In [1]:
# from moccasin import setup_notebook

# setup_notebook()

In [8]:
from moccasin.config import get_active_netwrok
active_netwrok = get_active_netwrok
print(active_netwrok)

ImportError: cannot import name 'get_active_netwrok' from 'moccasin.config' (c:\programming\algo-trading\Algo-Trading\MoxProjectAlgoTrading\.venv\Lib\site-packages\moccasin\config.py)

In [6]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa 

STARTING_ETH_BALANCE = int(1000e18)
STARTING_WETH_BALANCE = int(1e18)
STARTING_USDC_BALANCE = int(100e6)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth):
    print(f"Starting balance of the weth: {weth.balanceOf(boa.env.eoa)}")
    weth.deposit(value = STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)

    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print("After deposit:")
    print("ETH:", boa.env.get_balance(boa.env.eoa))
    print("WETH:", weth.balanceOf(boa.env.eoa))
    print("USDC:", usdc.balanceOf(boa.env.eoa))
    

def setup_script() -> Tuple[ABIContract, ABIContract,ABIContract,ABIContract]:
    print("Starting setup script...")

    active_netwrork = get_active_network()

    usdc = active_netwrork.manifest_named("usdc")
    weth = active_netwrork.manifest_named("weth")

    if active_netwrork.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc,weth)

def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script...


ValueError: Global Config object not initialized, initialize with initialize_global_config

In [ ]:
usdc = active_netwrork.manifest_named("usdc")
weth = active_netwrork.manifest_named("weth")

In [ ]:
from moccasin.config import get_config
config= get_config()
config.reload()
active_netwrork = config.get_active_network()
aavev3_pool_addresses_provider = active_netwrork.manifest_named("aavev3_pool_addresses_provider")
pool_address = aavev3_pool_addresses_provider.getPool()
print(pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [ ]:
config.reload()
active_netwrork = config.get_active_network()
pool_contract = active_netwrork.manifest_named("pool", address = pool_address)

In [ ]:
REFFERAL_CODE = 0
def deposit(pool_contract, token, amount):
    allowed_amout = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amout < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {token.name()} into Aave contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFFERAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)
if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(totalCollateralBase,   
totalDebtBase,   
availableBorrowsBase,
currentLiquidationThreshold, 
ltv,    
healthFactor) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"User Account Data \n totalCollateralBase = {totalCollateralBase}, \n totalDebtBase = {totalDebtBase}, \n availableBorrowsBase = {availableBorrowsBase} \n currentLiquidationThreshold = {currentLiquidationThreshold} \n ltv = {ltv} \n healthFactor = {healthFactor}")



User Account Data 
 totalCollateralBase = 795747514812, 
 totalDebtBase = 0, 
 availableBorrowsBase = 638905679643 
 currentLiquidationThreshold = 8281 
 ltv = 8029 
 healthFactor = 115792089237316195423570985008687907853269984665640564039457584007913129639935


In [ ]:
config.reload()
active_netwrork = config.get_active_network()
aave_protocol_data_provider = active_netwrork.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[getAllATokens(symbol='aEthWETH', tokenAddress=Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), getAllATokens(symbol='aEthwstETH', tokenAddress=Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), getAllATokens(symbol='aEthWBTC', tokenAddress=Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), getAllATokens(symbol='aEthUSDC', tokenAddress=Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), getAllATokens(symbol='aEthDAI', tokenAddress=Address('0x018008bfb33d285247A21d44E50697654f754e63')), getAllATokens(symbol='aEthLINK', tokenAddress=Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), getAllATokens(symbol='aEthAAVE', tokenAddress=Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), getAllATokens(symbol='aEthcbETH', tokenAddress=Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), getAllATokens(symbol='aEthUSDT', tokenAddress=Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), getAllATokens(symbol='aEthrETH', tokenAddress=Address('0xCc9EE9483f662091a1de47

In [ ]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_netwrork.manifest_named('usdc', address = a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_netwrork.manifest_named('usdc', address = a_token[1])

print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [ ]:
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)
print(a_usdc_balance / 1000000)
print(a_weth_balance / 1000000000000000000)

300.0
3.0
